In [1]:
import pandas as pd
import numpy as np

for position if there exists a higher value at same price remove
for position if there exists a lower price at same value

In [2]:
# Number of Random Players to Generate of form (Position, Value, Price) where Position is 1-5, Value is 1-51,
# and Price is 1-50 (but is 2/3rds based on Value)
n = 10000
pos = np.random.choice(range(1, 6), n)
value = np.random.choice(range(1, 52), n)
price = [2*x/3 + y/float(10) for x, y in zip(value, np.random.choice(range(1, 17), n))]

players = zip(pos, value, price)
#players = [(1,100,49.9), (1,1,9.1), (2,2,9.2), (3,3,9.3), (4,4,9.4), (5,91,9.5)]

In [3]:
# Set our Price Ceiling and subdivisions of a dollar, here we go by .1 or a tenth and create a price dictionary
price_ceiling = 50
density = 10
prices = {v : (0,[]) for v in range(price_ceiling*density+1)} 

In [4]:
# Create a player dictionary by position with the form {Pos : [(Val, Price), (V2,P2), ...]}
pos_d = {}
for p in players:
    try:
        pos_d[p[0]].append((p[1],int(p[2]*density)))
    except:
        pos_d[p[0]] = [(p[1],int(p[2]*density))]

In [5]:
# Print out number of players at each position
for k, v in pos_d.iteritems():
    print k, len(v)

1 2002
2 1996
3 2045
4 1962
5 1995


In [6]:
# These functions prune our frontier (of players p,v combos) by checking to see if they are strictly dominated.
# This means it checks to see if we can get the same value at a lower price, or higher value at the same price, we remove the node.
# This results in a strictly increasing relationship between price and value and could be graphed

def helper(val, d, up):
    if up:
        return max([v for k, v in d.iteritems() if k <= val])
    else:
        return min([v for k, v in d.iteritems() if k >= val])
        

def dominated(price, value):
    prices = np.unique(price)
    values = np.unique(value)
    best_vals = {p : max([value[i] for i, v in enumerate(price) if v == p]) for p in prices}
    best_pris = {v : min([price[i] for i, p in enumerate(value) if p == v]) for v in values}
    bests = [(k,v) for k, v in best_vals.iteritems()]+[(v,k) for k, v in best_pris.iteritems()]
    bests_ = [(p,v) for p, v in bests if (v >= helper(p, best_vals, True)) and (p <= helper(v, best_pris, False))]
    return set(bests_)

In [7]:
# This function checks for each value if it has the proper number of players, and replaces it with 0 if it is impossible (meaning 
# that there are no combos to be at or below that price), or replaces it with the highest achievable value at some lower price
# with the correct number of players.

def clean(d, val):
    lst = []
    for k, v in d.iteritems():
        if len(v[1]) == val+1:
            lst.append(v)
        else:
            try:
                d[k] = lst[-1]
            except IndexError:
                d[k] = (0,[])
    return d

In [8]:
#  This goes by position and creates new price, value dict.  Has the option to require a player is added from each position. 
def add_pos(in_d, vals, require=False):
    out_d = in_d.copy()
    for p, s in in_d.iteritems():
        for v, p_ in vals:
            if p+p_ <= max(in_d.keys()):
                if out_d[p+p_][0] < in_d[p][0]+v:
                    out_d[p+p_] = (in_d[p][0]+v, in_d[p][1]+[(v, p_)])
    if require:
        out_d = clean(out_d, val = len(in_d[max(in_d.keys())][1]))
    return out_d

In [9]:
in_d = prices.copy()
for p in pos_d.keys():
    in_d = add_pos(in_d, list(dominated(*zip(*pos_d[p]))), require=True)

In [10]:
in_d

{0: (0, []),
 1: (0, []),
 2: (0, []),
 3: (0, []),
 4: (0, []),
 5: (0, []),
 6: (0, []),
 7: (0, []),
 8: (0, []),
 9: (0, []),
 10: (0, []),
 11: (0, []),
 12: (0, []),
 13: (0, []),
 14: (0, []),
 15: (0, []),
 16: (0, []),
 17: (0, []),
 18: (0, []),
 19: (0, []),
 20: (0, []),
 21: (0, []),
 22: (0, []),
 23: (0, []),
 24: (0, []),
 25: (0, []),
 26: (0, []),
 27: (0, []),
 28: (0, []),
 29: (0, []),
 30: (0, []),
 31: (0, []),
 32: (0, []),
 33: (0, []),
 34: (0, []),
 35: (0, []),
 36: (0, []),
 37: (0, []),
 38: (0, []),
 39: (0, []),
 40: (0, []),
 41: (0, []),
 42: (0, []),
 43: (0, []),
 44: (0, []),
 45: (0, []),
 46: (0, []),
 47: (0, []),
 48: (0, []),
 49: (0, []),
 50: (0, []),
 51: (0, []),
 52: (0, []),
 53: (0, []),
 54: (0, []),
 55: (0, []),
 56: (0, []),
 57: (0, []),
 58: (0, []),
 59: (0, []),
 60: (0, []),
 61: (0, []),
 62: (0, []),
 63: (0, []),
 64: (0, []),
 65: (0, []),
 66: (0, []),
 67: (0, []),
 68: (0, []),
 69: (0, []),
 70: (0, []),
 71: (0, []),
 7